## Simplest neural network 
Single layer, (no hidden layers) using backprop

Original code from Andrew Trask

http://iamtrask.github.io/2015/07/12/basic-python-network/

Notebook version: Ravi Annaswamy March 30, 2017

In [1]:
import numpy as np
np.random.seed(1)

### X Matrix holds input test cases


Neural networks learn mappings from an input situation to an output action/response/judgement.

The mapping we want it to learn here is the following:

0,0,1 ->0

0,1,1 ->0

1,0,1 ->1

1,1,1 ->1

Basically column 3 has no effect on the output, and if you look closely column 2 also has no effect. Only column one directly drives output. Can our network learn this?

Let us create an input dataset with four rows (four cases) and three columns each

In [2]:
X = np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,1]])
X

array([[0, 0, 1],
       [0, 1, 1],
       [1, 0, 1],
       [1, 1, 1]])

### Y matrix holds output test cases

Corresponding outputs

In [3]:
y = np.array([[0,0,1,1]]).T

In [4]:
y

array([[0],
       [0],
       [1],
       [1]])

### The Weights Matrix will hold the knowledge

We will create a single layer network 3 input cells and one output cell, means that we
need a 3x1 weight matrix.

We call this as a single layer network, because there is one layer of weights to be learnt. We will call input layer as L0 and the output layer as L1.

We will use numpy random generator and ask for 3 rows of 1 weight each.
We will have these centered at 0 by initially generating from 0 to 2 and subtracting 1.

In [5]:
W0 = 2*np.random.random((3,1))-1

In [6]:
W0

array([[-0.16595599],
       [ 0.44064899],
       [-0.99977125]])

### Calculating output for a test case

Let us present the inputs to an input layer

In [7]:
L0 = X

The output is obtained by multiplying the inputs one row at a time with the matrix.


For instance, the first input case [0,0,1] is processed by these weights like this:
    

In [8]:
np.dot(X[0],W0)

array([-0.99977125])

### Batch Processing

We could do all input rows at the same time! We save the output as Layer 1, L1

In [9]:
L1 = np.dot(L0,W0); L1

array([[-0.99977125],
       [-0.55912226],
       [-1.16572724],
       [-0.72507825]])

This is cool because we can find our error against the four expected outputs stored in y

In [10]:
L1_error= y-L1; L1_error

array([[ 0.99977125],
       [ 0.55912226],
       [ 2.16572724],
       [ 1.72507825]])

For case 1, we expected the network to give out a 0, but since the fourth input was on and it passed along
its value, the output was -0.999, so we need to reduce the output to 0, and so the amount to change is 0.999

In [11]:
L1_delta = L1_error * L1; L1_delta

array([[-0.99954255],
       [-0.31261771],
       [-2.52464724],
       [-1.25081673]])

This is the direction in which we have to adjust the weights.

In [12]:
W0 += np.dot(L0.T, L1_delta)

In [13]:
W0

array([[-3.94141996],
       [-1.12278545],
       [-6.08739548]])

Has this improved outputs?

In [14]:
L1 = np.dot(L0, W0)

In [15]:
L1

array([[ -6.08739548],
       [ -7.21018093],
       [-10.02881544],
       [-11.15160089]])

No, we expected first two rows to move towards 0 and the other two to move towards 1, but that is not what we see.

Somehow we have not scaled the error into a proper feedback that will update weights just enough.

Maybe we need to run a few more rounds?


In [16]:
np.random.seed(1)
W0 = 2*np.random.random((3,1))-1
for iter in xrange(5):
        L0 = X
        L1 = np.dot(L0, W0)
        L1_error = y - L1
        L1_delta = L1_error * L1
        W0 +=  np.dot(L0.T, L1_delta)
        print(iter, L1)

(0, array([[-0.99977125],
       [-0.55912226],
       [-1.16572724],
       [-0.72507825]]))
(1, array([[ -6.08739548],
       [ -7.21018093],
       [-10.02881544],
       [-11.15160089]]))
(2, array([[-341.24624596],
       [-529.86554361],
       [-591.30342369],
       [-779.92272134]]))
(3, array([[-1356838.15721241],
       [-2246843.64480525],
       [-2316378.63067319],
       [-3206384.11826603]]))
(4, array([[ -2.25358321e+13],
       [ -3.78650417e+13],
       [ -3.81823477e+13],
       [ -5.35115572e+13]]))


Ouch! What happened here?

We see the outputs crazily shooting to large numbers!

# Enter the Sigmoid.


Very clever people, who had experience developing feedback control systems, got the insight into what is happening.

If you feed unscaled feedback, it becomes positive feedback and values begin to shoot out!

One solution is to use a multiplier like 0.01 to limit how much the weights will change each iteration. 

For this example, it will work, but in other cases, it will just postpone the out of control dance. What we really need is a way to ensure that the feedback signal will always be far less than one.

This is done by rescaling outputs themselves in the range [0-1].

The Sigmoid is a beautiful function that transforms a value ranging 
from negative infinity to positive infinity to range of [0-1].

Remember, we need to limit both the output and the feedback amount using this function.

First the function itself.

In [17]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
print(-20, sigmoid(-20))
print(100, sigmoid(100))

(-20, 2.0611536181902037e-09)
(100, 1.0)


Now, let us make the feedback proportional to the slope of the output.
In other words, if the output is large and negative, the feedback should be 
large and negative and so on.

Andy gives the beautiful intuition that the gradient shows how sure you are.

If x is 1, gradient will be 1 * (1-1) = 0
if x is 0, gradient will be 0 * (1-0) = 0
but if x is 0.5, gradient will be 0.5 * 0.5 = 0.25

In [30]:
def gradient(x):
    return x*(1-x)
print(0, gradient(0))
print(1, gradient(1))
print(0.5, gradient(0.5))

(0, 0)
(1, 0)
(0.5, 0.25)


Let us redo the forward computation and error delta with this new scheme.

Since we have updated W0 once or twice above, I am resetting to initial known seed state in the first two lines.

In [31]:
np.random.seed(1)
W0 = 2*np.random.random((3,1))-1

L0 = X
L1 = sigmoid(np.dot(L0, W0))
print('L1: ',L1)

L1_error = y - L1
print('Error: ', L1_error)

L1_gradient = gradient(L1)
print('L1 gradient:', L1_gradient)

L1_delta = L1_error * L1_gradient
print('L1_delta: ', L1_delta)

('L1: ', array([[ 0.2689864 ],
       [ 0.36375058],
       [ 0.23762817],
       [ 0.3262757 ]]))
('Error: ', array([[-0.2689864 ],
       [-0.36375058],
       [ 0.76237183],
       [ 0.6737243 ]]))
('L1 gradient:', array([[ 0.19663272],
       [ 0.23143609],
       [ 0.18116102],
       [ 0.21981987]]))
('L1_delta: ', array([[-0.05289153],
       [-0.08418501],
       [ 0.13811206],
       [ 0.14809799]]))


Now we are ready to update weights, but by how much? We use the delta rule, that says weight update should depend on how high an input was and how much the error was! Makes sense.

In [32]:
np.dot(L0.T, L1_delta)

array([[ 0.28621005],
       [ 0.06391297],
       [ 0.14913351]])

Let us do that.

In [21]:
W0 += np.dot(L0.T, L1_delta)
W0

array([[ 0.12025406],
       [ 0.50456196],
       [-0.85063774]])

Let us see if this will improve output next time.

In [22]:
L1 = sigmoid(np.dot(L0, W0))
L1

array([[ 0.29929909],
       [ 0.41433436],
       [ 0.32511054],
       [ 0.44378327]])

The zeroth iteration, the output was:

    array([[-0.99977125],
       [-0.55912226],
       [-1.16572724],
       [-0.72507825]])
    
    Now it is:
        array([[ 0.29929909],
       [ 0.41433436],
       [ 0.32511054],
       [ 0.44378327]])
        
      

So it looks like the outputs are moving in the right direction, though
        they seem to have overshot beyond 0..
        
Let us try more iterations, now that we have found a way to 
tame the feedback.


In [26]:
np.random.seed(1)
W0 = 2*np.random.random((3,1))-1
for iter in xrange(10000):
        L0 = X
        L1 = sigmoid(np.dot(L0, W0))
        L1_error = y - L1
        L1_delta = L1_error * gradient(L1)
        W0 += np.dot(L0.T, L1_delta)
print(iter, L1)

(9999, array([[ 0.00966449],
       [ 0.00786506],
       [ 0.99358898],
       [ 0.99211957]]))


Wow! Success!

## Entire program
So here is the entire program for clarity..

Please note that I have made two final changes.
One, I have moved the seed reset command to reset the random seed just before we sample the initial weights. 

Second, I am  now printing results once every 1000 iterations.


In [33]:
import numpy as np

X = np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,1]])
y = np.array([[0,0,1,1]]).T

np.random.seed(1)
W0 = 2*np.random.random((3,1))-1
for iter in xrange(10000):
        L0 = X
        L1 = sigmoid(np.dot(L0, W0))
        L1_error = y - L1
        L1_delta = L1_error * gradient(L1)
        W0 += np.dot(L0.T, L1_delta)
        if iter%1000==0:
            print(iter, L1)

(0, array([[ 0.2689864 ],
       [ 0.36375058],
       [ 0.23762817],
       [ 0.3262757 ]]))
(1000, array([[ 0.03176745],
       [ 0.02575143],
       [ 0.97907779],
       [ 0.97416005]]))
(2000, array([[ 0.02210122],
       [ 0.01793507],
       [ 0.985409  ],
       [ 0.98200541]]))
(3000, array([[ 0.0179128 ],
       [ 0.01454746],
       [ 0.98815758],
       [ 0.98540875]]))
(4000, array([[ 0.01544409],
       [ 0.0125494 ],
       [ 0.98978021],
       [ 0.98741602]]))
(5000, array([[ 0.01377152],
       [ 0.01119485],
       [ 0.99088089],
       [ 0.98877656]]))
(6000, array([[ 0.01254323],
       [ 0.01019961],
       [ 0.99168995],
       [ 0.98977602]]))
(7000, array([[ 0.01159234],
       [ 0.0094288 ],
       [ 0.99231678],
       [ 0.99054995]]))
(8000, array([[ 0.01082824],
       [ 0.00880917],
       [ 0.99282079],
       [ 0.99117199]]))
(9000, array([[ 0.01019693],
       [ 0.00829707],
       [ 0.99323743],
       [ 0.991686  ]]))


Before we move off, is this network all powerful?

Let us try this mapping:
    
    0,0,1 -> 0
    0,1,1 -> 1
    1,0,1 -> 1
    1,1,1 -> 1
    
    which is an OR mapping on columns one and two, and third column is just noise with no information to predict output. 

In [36]:
import numpy as np

X = np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,1]])
y = np.array([[0,1,1,1]]).T

np.random.seed(1)
W0 = 2*np.random.random((3,1))-1
for iter in xrange(10000):
        L0 = X
        L1 = sigmoid(np.dot(L0, W0))
        L1_error = y - L1
        L1_delta = L1_error * gradient(L1)
        W0 += np.dot(L0.T, L1_delta)
        if iter%1000==0:
            print(iter, L1)

(0, array([[ 0.2689864 ],
       [ 0.36375058],
       [ 0.23762817],
       [ 0.3262757 ]]))
(1000, array([[ 0.05479081],
       [ 0.96577999],
       [ 0.96577641],
       [ 0.99992722]]))
(2000, array([[ 0.03773302],
       [ 0.97633606],
       [ 0.97633549],
       [ 0.99997696]]))
(3000, array([[ 0.03044286],
       [ 0.98087556],
       [ 0.98087536],
       [ 0.99998806]]))
(4000, array([[ 0.02617603],
       [ 0.98353992],
       [ 0.98353983],
       [ 0.99999247]]))
(5000, array([[ 0.0232981 ],
       [ 0.98534006],
       [ 0.98534001],
       [ 0.99999472]]))
(6000, array([[ 0.02119131],
       [ 0.98665939],
       [ 0.98665936],
       [ 0.99999604]]))
(7000, array([[ 0.01956422],
       [ 0.9876792 ],
       [ 0.98767918],
       [ 0.99999689]]))
(8000, array([[ 0.01825921],
       [ 0.98849769],
       [ 0.98849768],
       [ 0.99999748]]))
(9000, array([[ 0.01718266],
       [ 0.98917326],
       [ 0.98917325],
       [ 0.99999791]]))


Yay! It did. Let us try the and mapping
    

In [40]:
import numpy as np

X = np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,1]])
y = np.array([[0,0,0,1]]).T

np.random.seed(1)
W0 = 2*np.random.random((3,1))-1
for iter in xrange(10000):
        L0 = X
        L1 = sigmoid(np.dot(L0, W0))
        L1_error = y - L1
        L1_delta = L1_error * gradient(L1)
        W0 += np.dot(L0.T, L1_delta)
        if iter%1000==0:
            print(iter, L1)

(0, array([[ 0.2689864 ],
       [ 0.36375058],
       [ 0.23762817],
       [ 0.3262757 ]]))
(1000, array([[  2.43631161e-04],
       [  5.54627689e-02],
       [  5.54627683e-02],
       [  9.33989117e-01]]))
(2000, array([[  7.45477397e-05],
       [  3.81302776e-02],
       [  3.81302776e-02],
       [  9.54707154e-01]]))
(3000, array([[  3.80129661e-05],
       [  3.07209321e-02],
       [  3.07209321e-02],
       [  9.63537553e-01]]))
(4000, array([[  2.37428297e-05],
       [  2.63893749e-02],
       [  2.63893749e-02],
       [  9.68693007e-01]]))
(5000, array([[  1.65408129e-05],
       [  2.34709841e-02],
       [  2.34709841e-02],
       [  9.72163723e-01]]))
(6000, array([[  1.23371120e-05],
       [  2.13365563e-02],
       [  2.13365563e-02],
       [  9.74700716e-01]]))
(7000, array([[  9.64126894e-06],
       [  1.96894054e-02],
       [  1.96894054e-02],
       [  9.76657736e-01]]))
(8000, array([[  7.79438364e-06],
       [  1.83691746e-02],
       [  1.83691746e-02],

yes, it does.

How about the XOR mapping which detects the condition where
the output turns on only when ONE of the two inputs is on, but not whe
both are on!

That is, we want:
    
0,0,1->0 

0,1,1->1 

1,0,1->1 

1,1,1->0


In [41]:
import numpy as np

X = np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,1]])
y = np.array([[0,1,1,0]]).T

np.random.seed(1)
W0 = 2*np.random.random((3,1))-1
for iter in xrange(10000):
        L0 = X
        L1 = sigmoid(np.dot(L0, W0))
        L1_error = y - L1
        L1_delta = L1_error * gradient(L1)
        W0 += np.dot(L0.T, L1_delta)
        if iter%1000==0:
            print(iter, L1)

(0, array([[ 0.2689864 ],
       [ 0.36375058],
       [ 0.23762817],
       [ 0.3262757 ]]))
(1000, array([[ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5]]))
(2000, array([[ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5]]))
(3000, array([[ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5]]))
(4000, array([[ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5]]))
(5000, array([[ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5]]))
(6000, array([[ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5]]))
(7000, array([[ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5]]))
(8000, array([[ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5]]))
(9000, array([[ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5]]))


WHAT!

No it did not.

Turns out it CANNOT. A network without an intermediate 'thinking' layer
layer cannot learn this mapping. 

Why? You have to first process the inputs to see there is an OR condition, 
and separately to see if there is an AND condition. This should happen in layer 1 and 
another layer should look at this to determine the possibilities.

That is subject for the next notebook.
